Prepare the data:

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import torch.optim as optim
import torchvision.datasets as datasets
from torch.autograd import Variable
#from torchsummary import summary

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.is_available()

cuda:0


True

In [3]:
class CIFAR10RandomLabels(datasets.CIFAR10):
    """CIFAR10 dataset, with support for randomly corrupt labels.
    Params
    ------  
    corrupt_prob: float
    Default 0.0. The probability of a label being replaced with
    random label.
    num_classes: int
    Default 10. The number of classes in the dataset.
    """
    def __init__(self, corrupt_prob=0.0, num_classes=10, **kwargs):
        super(CIFAR10RandomLabels, self).__init__(**kwargs)
        self.n_classes = num_classes
        if corrupt_prob > 0:
            self.corrupt_labels(corrupt_prob)

    def corrupt_labels(self, corrupt_prob):
        labels = np.array(self.targets)
        np.random.seed(12345)
        mask = np.random.rand(len(labels)) <= corrupt_prob
        rnd_labels = np.random.choice(self.n_classes, mask.sum())
        labels[mask] = rnd_labels
        # we need to explicitly cast the labels from npy.int64 to
        # builtin int type, otherwise pytorch will fail...
        labels = [int(x) for x in labels]
        self.targets = torch.tensor(labels)

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainloader = torch.utils.data.DataLoader(
        CIFAR10RandomLabels(root='./data', train=True, download=True, 
                            transform=transform, corrupt_prob=0.2),
                            batch_size=128, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(
        CIFAR10RandomLabels(root='./data', train=False, download=True,
                            transform=transform, corrupt_prob=0.2),
                            batch_size=100, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class MaskedLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super(MaskedLinear, self).__init__(in_features, out_features, bias)
        self.mask_flag = False
    
    def set_mask(self, mask):
        self.mask = Variable(mask, requires_grad=False)
        self.weight.data = self.weight.data*self.mask.data
        self.mask_flag = True
    
    def get_mask(self):
        print(self.mask_flag)
        return self.mask
    
    def prune(self, threshold):
        # generate mask
        for params in self.parameters():
            if len(params.data.size()) != 1: # Not bias
                mask = params.data.abs() > threshold
                self.set_mask(mask.float()) 
    
    def forward(self, x):
        if self.mask_flag == True:
            weight = self.weight*self.mask
            return F.linear(x, weight, self.bias)
        else:
            return F.linear(x, self.weight, self.bias)
        

        
class MaskedConv2d(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        super(MaskedConv2d, self).__init__(in_channels, out_channels, 
            kernel_size, stride, padding, dilation, groups, bias)
        self.mask_flag = False
    
    def set_mask(self, mask):
        self.mask = Variable(mask, requires_grad=False)
        self.weight.data = self.weight.data*self.mask.data
        self.mask_flag = True
    
    def get_mask(self):
        print(self.mask_flag)
        return self.mask
    
    def prune(self, threshold):
        # generate mask
        for params in self.parameters():
            if len(params.data.size()) != 1: # Not bias
                mask = params.data.abs() > threshold
                self.set_mask(mask.float()) 
    
    def forward(self, x):
        if self.mask_flag == True:
            weight = self.weight*self.mask
            return F.conv2d(x, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
        else:
            return F.conv2d(x, self.weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

Define the network:

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        linear = MaskedLinear
        conv2d = MaskedConv2d
        self.conv1 = conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = conv2d(6, 16, 5)
        self.fc1 = linear(16 * 5 * 5, 240)
        self.fc2 = linear(240, 120)
        self.fc3 = linear(120, 84)
        self.fc4 = linear(84, 10)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
    def weight_prune(self, pruning_perc):
        all_weights = []
        for params in self.parameters():
            if len(params.data.size()) != 1:
                all_weights += list(params.cpu().data.abs().numpy().flatten())
        threshold = np.percentile(np.array(all_weights), pruning_perc)
        print(f'Pruning with threshold : %.4f' % threshold)
        
        # Module here refers to layer
        for name, module in self.named_modules():
            if name in ['conv1','conv2','fc1','fc2','fc3','fc4']:
                module.prune(threshold)                

net = Net()
net.to(device)
#summary(net, (3, 32, 32))

Net(
  (conv1): MaskedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): MaskedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): MaskedLinear(in_features=400, out_features=240, bias=True)
  (fc2): MaskedLinear(in_features=240, out_features=120, bias=True)
  (fc3): MaskedLinear(in_features=120, out_features=84, bias=True)
  (fc4): MaskedLinear(in_features=84, out_features=10, bias=True)
)

In [7]:
def train(model, optimizer, epochs, early_stop_step=0):
    step_counter = 0
    best_accuracy = 0
    for epoch in range(epochs):  # loop over the dataset multiple times
      
        train_correct = 0
        train_total = 0
        running_loss = 0.0
        train_accuracy = 0.0
        
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)
    
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 100 mini-batches
             #   print('[%d, %5d] loss: %.3f' %
             #         (epoch + 1, i + 1, running_loss / 100))
                train_loss = running_loss / 100
                running_loss = 0.0

        if epoch % 10 == 9:     
            with torch.no_grad():
                for data in trainloader:
                    inputs, labels = data[0].to(device), data[1].to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                    
            train_accuracy = 100 * train_correct / train_total
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, train_loss))
            print('Train Accuracy: %.3f %%' % train_accuracy)
            
        '''
        if early_stop_step != 0:
            step_counter += 1
            if best_accuracy < train_accuracy:
                best_accuracy = train_accuracy
                step_counter = 0
            if step_counter == early_stop_step or epoch == epochs - 1:
                print("Early stop at epoch %d" % (epoch + 1))
                print('Train Accuracy: %.3f %%, Train Loss: %.3f' % (train_accuracy, train_loss))
                break
        '''
        
        if train_accuracy - 100.0 < 0.001 and train_loss - 0.0 < 0.001:
            print("Reach 100 %% train accuracy at epoch %d" % (epoch + 1))
            break
            
    #print('Finished Training')

In [8]:
def test(model):
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss += criterion(outputs, labels).item()
    
    #print('Test Accuracy on 10000 images: %.2f %%' % (
    #    100 * correct / total))
    
    # test loss for 100 mini-batches)
    print('Test Accuracy: %.3f %%, Test Loss: %.3f' % (100 * correct / total, test_loss / 100))

In [9]:
def calculate_size(model):
    non_zeros = 0
    for param in model.parameters():
        if param.requires_grad==True:
            non_zeros += param.nonzero().size(0)
    params_size = non_zeros* 4. / (1024 ** 2)
    print("Params size (MB): %0.3f" % params_size)

In [10]:
calculate_size(net)
criterion = nn.CrossEntropyLoss() # Softmax is built in it so you do not need add that on the last layer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#initial_optimizer_state_dict = optimizer.state_dict()
train(net, optimizer, 5000)
model_path = "models/LeNet_480_84_10_random100.pt"
torch.save(net.state_dict(), model_path)
test(net)
calculate_size(net)

Params size (MB): 0.530
[10,   391] loss: 2.188
Train Accuracy: 21.054 %
[20,   391] loss: 1.871
Train Accuracy: 35.784 %
[30,   391] loss: 1.719
Train Accuracy: 42.478 %
[40,   391] loss: 1.634
Train Accuracy: 47.622 %
[50,   391] loss: 1.562
Train Accuracy: 51.482 %
[60,   391] loss: 1.473
Train Accuracy: 54.442 %
[70,   391] loss: 1.394
Train Accuracy: 56.314 %
[80,   391] loss: 1.325
Train Accuracy: 61.142 %
[90,   391] loss: 1.216
Train Accuracy: 63.840 %
[100,   391] loss: 1.095
Train Accuracy: 67.452 %
[110,   391] loss: 1.016
Train Accuracy: 70.966 %
[120,   391] loss: 0.869
Train Accuracy: 73.162 %
[130,   391] loss: 0.752
Train Accuracy: 76.296 %
[140,   391] loss: 0.637
Train Accuracy: 81.128 %
[150,   391] loss: 0.518
Train Accuracy: 83.382 %
[160,   391] loss: 0.399
Train Accuracy: 87.438 %
[170,   391] loss: 0.343
Train Accuracy: 90.920 %
[180,   391] loss: 0.312
Train Accuracy: 90.286 %
[190,   391] loss: 0.245
Train Accuracy: 92.468 %
[200,   391] loss: 0.235
Train Accu

In [ ]:
criterion = nn.CrossEntropyLoss()
model_path = "models/LeNet_480_84_10_random100.pt"
parameter_size = [0.230, 0.184, 0.139, 0.093, 0.047]
percent = [(1 - x / 0.530) * 100 for x in parameter_size]
for prune_percent in percent:
    net_p = Net()
    net_p.load_state_dict(torch.load(model_path))
    net_p.to(device)
    net_p.eval()
    print("--------------------------------------")
    print("Prune the model by %.2f %%" % prune_percent)
    net_p.weight_prune(prune_percent)
    #summary(net, (3, 32, 32))
    calculate_size(net_p)
    optimizer = optim.SGD(net_p.parameters(), lr=0.001, momentum=0.9)
    train(net_p, optimizer, 6000, early_stop_step=0)
    test(net_p)

--------------------------------------
Prune the model by 56.60 %
Pruning with threshold : 0.0587
Params size (MB): 0.231
[10,   391] loss: 0.366
Train Accuracy: 88.806 %
[20,   391] loss: 0.224
Train Accuracy: 92.828 %
[30,   391] loss: 0.177
Train Accuracy: 95.464 %
[40,   391] loss: 0.128
Train Accuracy: 96.094 %
[50,   391] loss: 0.133
Train Accuracy: 96.666 %
[60,   391] loss: 0.131
Train Accuracy: 93.942 %
[70,   391] loss: 0.142
Train Accuracy: 96.138 %
[80,   391] loss: 0.117
Train Accuracy: 97.114 %
[90,   391] loss: 0.008
Train Accuracy: 99.996 %
[100,   391] loss: 0.004
Train Accuracy: 99.998 %
[110,   391] loss: 0.003
Train Accuracy: 100.000 %
[120,   391] loss: 0.002
Train Accuracy: 100.000 %
[130,   391] loss: 0.002
Train Accuracy: 100.000 %
[140,   391] loss: 0.001
Train Accuracy: 100.000 %
[150,   391] loss: 0.001
Train Accuracy: 100.000 %
[160,   391] loss: 0.001
Train Accuracy: 100.000 %
[170,   391] loss: 0.001
Train Accuracy: 100.000 %
Reach 100 % train accuracy at 

[980,   391] loss: 0.268
Train Accuracy: 91.480 %
[990,   391] loss: 0.253
Train Accuracy: 91.004 %
[1000,   391] loss: 0.252
Train Accuracy: 92.296 %
[1010,   391] loss: 0.250
Train Accuracy: 92.884 %
[1020,   391] loss: 0.275
Train Accuracy: 90.944 %
[1030,   391] loss: 0.296
Train Accuracy: 90.922 %
[1040,   391] loss: 0.249
Train Accuracy: 91.560 %
[1050,   391] loss: 0.276
Train Accuracy: 90.842 %
[1060,   391] loss: 0.289
Train Accuracy: 90.488 %
[1070,   391] loss: 0.249
Train Accuracy: 92.862 %
[1080,   391] loss: 0.277
Train Accuracy: 90.678 %
[1090,   391] loss: 0.246
Train Accuracy: 92.724 %
[1100,   391] loss: 0.254
Train Accuracy: 92.160 %
[1110,   391] loss: 0.262
Train Accuracy: 91.566 %
[1120,   391] loss: 0.260
Train Accuracy: 92.126 %
[1130,   391] loss: 0.252
Train Accuracy: 91.232 %
[1140,   391] loss: 0.261
Train Accuracy: 91.868 %
[1150,   391] loss: 0.228
Train Accuracy: 92.428 %
[1160,   391] loss: 0.254
Train Accuracy: 93.490 %
[1170,   391] loss: 0.251
Train A

[2590,   391] loss: 0.242
Train Accuracy: 92.128 %
[2600,   391] loss: 0.271
Train Accuracy: 91.244 %
[2610,   391] loss: 0.242
Train Accuracy: 93.082 %
[2620,   391] loss: 0.190
Train Accuracy: 92.358 %
[2630,   391] loss: 0.289
Train Accuracy: 93.116 %
[2640,   391] loss: 0.213
Train Accuracy: 91.782 %
[2650,   391] loss: 0.216
Train Accuracy: 91.896 %
[2660,   391] loss: 0.230
Train Accuracy: 92.920 %
[2670,   391] loss: 0.190
Train Accuracy: 94.580 %
[2680,   391] loss: 0.191
Train Accuracy: 93.966 %
[2690,   391] loss: 0.237
Train Accuracy: 93.188 %
[2700,   391] loss: 0.251
Train Accuracy: 93.532 %
[2710,   391] loss: 0.202
Train Accuracy: 93.528 %
[2720,   391] loss: 0.154
Train Accuracy: 94.088 %
[2730,   391] loss: 0.182
Train Accuracy: 94.872 %
[2740,   391] loss: 0.191
Train Accuracy: 95.418 %
[2750,   391] loss: 0.275
Train Accuracy: 92.428 %
[2760,   391] loss: 0.220
Train Accuracy: 92.094 %
[2770,   391] loss: 0.222
Train Accuracy: 94.446 %
[2780,   391] loss: 0.223
Train

[4200,   391] loss: 0.182
Train Accuracy: 93.480 %
[4210,   391] loss: 0.230
Train Accuracy: 94.164 %
[4220,   391] loss: 0.172
Train Accuracy: 94.304 %
[4230,   391] loss: 0.221
Train Accuracy: 94.054 %
[4240,   391] loss: 0.233
Train Accuracy: 91.192 %
[4250,   391] loss: 0.172
Train Accuracy: 92.704 %
[4260,   391] loss: 0.252
Train Accuracy: 91.130 %
[4270,   391] loss: 0.157
Train Accuracy: 94.680 %
[4280,   391] loss: 0.206
Train Accuracy: 92.976 %
[4290,   391] loss: 0.171
Train Accuracy: 93.504 %
[4300,   391] loss: 0.170
Train Accuracy: 92.790 %
[4310,   391] loss: 0.184
Train Accuracy: 94.396 %
[4320,   391] loss: 0.216
Train Accuracy: 92.926 %
[4330,   391] loss: 0.217
Train Accuracy: 94.386 %
[4340,   391] loss: 0.227
Train Accuracy: 95.250 %
[4350,   391] loss: 0.200
Train Accuracy: 90.920 %
[4360,   391] loss: 0.237
Train Accuracy: 93.676 %
[4370,   391] loss: 0.126
Train Accuracy: 92.062 %
[4380,   391] loss: 0.235
Train Accuracy: 93.184 %
[4390,   391] loss: 0.142
Train

[5810,   391] loss: 0.240
Train Accuracy: 94.022 %
[5820,   391] loss: 0.204
Train Accuracy: 93.388 %
[5830,   391] loss: 0.215
Train Accuracy: 93.782 %
[5840,   391] loss: 0.212
Train Accuracy: 91.104 %
[5850,   391] loss: 0.205
Train Accuracy: 94.378 %
[5860,   391] loss: 0.187
Train Accuracy: 93.402 %
[5870,   391] loss: 0.183
Train Accuracy: 95.428 %
[5880,   391] loss: 0.165
Train Accuracy: 94.986 %
[5890,   391] loss: 0.207
Train Accuracy: 95.188 %
[5900,   391] loss: 0.153
Train Accuracy: 92.240 %
[5910,   391] loss: 0.215
Train Accuracy: 92.640 %
[5920,   391] loss: 0.270
Train Accuracy: 89.538 %
[5930,   391] loss: 0.253
Train Accuracy: 93.202 %
[5940,   391] loss: 0.185
Train Accuracy: 93.860 %
[5950,   391] loss: 0.238
Train Accuracy: 94.440 %
[5960,   391] loss: 0.253
Train Accuracy: 94.000 %
[5970,   391] loss: 0.140
Train Accuracy: 93.558 %
[5980,   391] loss: 0.287
Train Accuracy: 93.216 %
[5990,   391] loss: 0.205
Train Accuracy: 91.674 %
[6000,   391] loss: 0.179
Train

...

[4630,   391] loss: 0.650
Train Accuracy: 79.642 %
[4640,   391] loss: 0.628
Train Accuracy: 78.392 %
[4650,   391] loss: 0.663
Train Accuracy: 79.956 %
[4660,   391] loss: 0.649
Train Accuracy: 79.798 %
[4670,   391] loss: 0.666
Train Accuracy: 79.086 %
[4680,   391] loss: 0.636
Train Accuracy: 79.848 %
[4690,   391] loss: 0.653
Train Accuracy: 78.794 %
[4700,   391] loss: 0.655
Train Accuracy: 79.198 %
[4710,   391] loss: 0.674
Train Accuracy: 79.716 %
[4720,   391] loss: 0.642
Train Accuracy: 78.026 %
[4730,   391] loss: 0.655
Train Accuracy: 79.822 %
[4740,   391] loss: 0.647
Train Accuracy: 79.032 %
[4750,   391] loss: 0.667
Train Accuracy: 78.760 %
[4760,   391] loss: 0.651
Train Accuracy: 79.444 %
[4770,   391] loss: 0.654
Train Accuracy: 79.776 %
[4780,   391] loss: 0.633
Train Accuracy: 79.986 %
[4790,   391] loss: 0.668
Train Accuracy: 79.672 %
[4800,   391] loss: 0.661
Train Accuracy: 78.864 %
[4810,   391] loss: 0.651
Train Accuracy: 80.084 %
[4820,   391] loss: 0.650
Train

[210,   391] loss: 1.399
Train Accuracy: 57.544 %
[220,   391] loss: 1.376
Train Accuracy: 57.822 %
[230,   391] loss: 1.367
Train Accuracy: 57.824 %
[240,   391] loss: 1.349
Train Accuracy: 57.882 %
[250,   391] loss: 1.342
Train Accuracy: 58.182 %
[260,   391] loss: 1.327
Train Accuracy: 58.388 %
[270,   391] loss: 1.348
Train Accuracy: 58.474 %
[280,   391] loss: 1.338
Train Accuracy: 58.626 %
[290,   391] loss: 1.340
Train Accuracy: 58.672 %
[300,   391] loss: 1.348
Train Accuracy: 58.588 %
[310,   391] loss: 1.326
Train Accuracy: 58.984 %
[320,   391] loss: 1.343
Train Accuracy: 58.960 %
[330,   391] loss: 1.315
Train Accuracy: 59.162 %
[340,   391] loss: 1.328
Train Accuracy: 59.278 %
[350,   391] loss: 1.325
Train Accuracy: 59.418 %
[360,   391] loss: 1.312
Train Accuracy: 59.366 %
[370,   391] loss: 1.324
Train Accuracy: 59.588 %
[380,   391] loss: 1.327
Train Accuracy: 59.638 %
[390,   391] loss: 1.303
Train Accuracy: 59.782 %
[400,   391] loss: 1.310
Train Accuracy: 59.682 %


[1840,   391] loss: 1.179
Train Accuracy: 63.164 %
[1850,   391] loss: 1.188
Train Accuracy: 62.996 %
[1860,   391] loss: 1.202
Train Accuracy: 63.132 %
[1870,   391] loss: 1.180
Train Accuracy: 63.076 %
[1880,   391] loss: 1.186
Train Accuracy: 62.966 %
[1890,   391] loss: 1.183
Train Accuracy: 62.992 %
[1900,   391] loss: 1.175
Train Accuracy: 63.240 %
[1910,   391] loss: 1.188
Train Accuracy: 62.996 %
[1920,   391] loss: 1.184
Train Accuracy: 63.146 %
[1930,   391] loss: 1.197
Train Accuracy: 63.126 %
[1940,   391] loss: 1.198
Train Accuracy: 62.942 %
[1950,   391] loss: 1.182
Train Accuracy: 62.980 %
[1960,   391] loss: 1.190
Train Accuracy: 62.984 %
[1970,   391] loss: 1.187
Train Accuracy: 63.212 %
[1980,   391] loss: 1.182
Train Accuracy: 63.140 %
[1990,   391] loss: 1.185
Train Accuracy: 63.192 %
[2000,   391] loss: 1.187
Train Accuracy: 63.096 %
[2010,   391] loss: 1.182
Train Accuracy: 63.222 %
[2020,   391] loss: 1.179
Train Accuracy: 63.316 %
[2030,   391] loss: 1.172
Train

[3450,   391] loss: 1.165
Train Accuracy: 63.974 %
[3460,   391] loss: 1.144
Train Accuracy: 63.990 %
[3470,   391] loss: 1.158
Train Accuracy: 63.800 %
[3480,   391] loss: 1.168
Train Accuracy: 63.940 %
[3490,   391] loss: 1.162
Train Accuracy: 63.694 %
[3500,   391] loss: 1.170
Train Accuracy: 63.940 %
[3510,   391] loss: 1.150
Train Accuracy: 63.956 %
[3520,   391] loss: 1.158
Train Accuracy: 63.810 %
[3530,   391] loss: 1.151
Train Accuracy: 63.894 %
[3540,   391] loss: 1.154
Train Accuracy: 63.740 %
[3550,   391] loss: 1.170
Train Accuracy: 64.038 %
[3560,   391] loss: 1.151
Train Accuracy: 63.848 %
[3570,   391] loss: 1.158
Train Accuracy: 63.794 %
[3580,   391] loss: 1.162
Train Accuracy: 63.998 %
[3590,   391] loss: 1.163
Train Accuracy: 63.896 %
[3600,   391] loss: 1.161
Train Accuracy: 63.802 %
[3610,   391] loss: 1.158
Train Accuracy: 63.858 %
[3620,   391] loss: 1.155
Train Accuracy: 63.816 %
[3630,   391] loss: 1.155
Train Accuracy: 63.988 %
[3640,   391] loss: 1.168
Train

[5060,   391] loss: 1.143
Train Accuracy: 63.950 %
[5070,   391] loss: 1.150
Train Accuracy: 64.134 %
[5080,   391] loss: 1.142
Train Accuracy: 64.156 %
[5090,   391] loss: 1.155
Train Accuracy: 64.120 %
[5100,   391] loss: 1.148
Train Accuracy: 64.302 %
[5110,   391] loss: 1.153
Train Accuracy: 64.354 %
[5120,   391] loss: 1.177
Train Accuracy: 64.056 %


In [ ]:
    test(net_p)